# Guidance
This script will 
- Data processing
- Model validation: creating TS/SP to constrain model simualtions driven by different mechanisms for each flight;
    - VOCs: HCHO, ALD2, RCHO, CHOCHO, Methylglyoxal, Glycoaldehyde, Formic acid, Acetic acid, Acetone, MEK, Isoprene, Monoterpenes, Methacrolein, MVK, Furans, and Maleic Anhydride;
    - NOy: NOx + HNO3 + HONO + PAN + MPAN + PPN + particulate nitrates;
- Data saving: Saving out the model outputs for each specie in corresponding directory;
- Unlike WE-CAN, FIREX lab only contains **met** data and **smk** concentration. The **bkg** concentration is based on assumptions. 

In [ ]:
# ==============
# Import modules
# ==============
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io

warnings.filterwarnings('ignore')

# change working directory, need to be customized
import os

os.chdir('/glade/u/home/lixujin/matlab/F0AM-4.2.1/Setups/Examples/Lixu/FIREX_LAB/output_data/')

### Functions

In [ ]:
# =========
# Functions
# =========
# design function to load table variable from MAT file.
def getF0AM_mat_conc(F0AM_model_file, variable_name):
    """
    read a struct-ified table variable (and column names) from a MAT-file
    and return pandas.DataFrame object.
    """
 
    # load file
    mod_struc = scipy.io.loadmat(F0AM_model_file)
    # get concentration from model
    mod_conc = (mod_struc['S']['Conc']).item().item()
    # get cnames
    mod_cnames = mod_struc['S']['Cnames'].item()
    # get the mdoel time (in sec)
    mod_time = mod_struc['S']['Time'].item()
    # convert tuple into array
    mod_conc = np.array(mod_conc)
    # convert 3D into 2D
    mod_conc = mod_conc[:,:,0]
    # make the data frame
    mod_df = pd.DataFrame(np.hstack((mod_conc, mod_cnames)))
    # save last column of dataframe as a series
    last_column = mod_df.iloc[: , -1]
    # delete the last column
    mod_df = mod_df.iloc[:, :-1]
    # get names from the model
    mod_cnames = []
    for data in last_column:
        data = data.item()
        mod_cnames.append(data)
    # rename index of the python dataframe
    mod_df.index = mod_cnames
    # rename the column names as time
    mod_time = mod_time[:,0]
    mod_df.columns = mod_time/60.0 # change second to minutes, for FIREX
    return mod_df.loc[variable_name,:]

def getF0AM_time(F0AM_model_file):
    # load file
    mod_struc = scipy.io.loadmat(F0AM_model_file)
    # get the mdoel time (in sec)
    mod_time = mod_struc['S']['Time'].item()
    return np.hstack(mod_time)

In [ ]:
# Function to read conc or bkg for MCM and GEOS-Chem F0AM output
# the get_bkg is obsolete for now
def get_df_specs(file_name, spec_list, get_conc, get_bkg, GC_setting, MCM_setting, obs_setting):
    # create a dummy dataframe
    spec_conc = getF0AM_mat_conc(file_name, 'CO') if get_conc==True else getF0AM_mat_bkg(file_name, 'CO')
    time = spec_conc.index
    data_dummy = [0]*len(time)
    df_dummy = pd.DataFrame(data_dummy, index = time, columns = ['Dummy'])

    for ind, spec in enumerate(spec_list):
        # =================================================
        # spec name adapted for different mech (customized)
        # =================================================
        if GC_setting==True:
            # map to the GC mech name, customized
            # Aldehydes
            if spec == 'Formaldehyde':                     spec = 'CH2O'
            if spec == 'Acetaldehyde':                     spec = 'ALD2'            
            #if spec == 'Lumped C>=3 aldehydes':            spec = 'RCHO'
            if spec == 'Glyoxal':                          spec = 'GLYX'
            if spec == 'Methylglyoxal':                    spec = 'MGLY'
            if spec == 'Glycoaldehyde':                    spec = 'GLYC'
            # Organic acids
            if spec == 'Formic acid':                      spec = 'HCOOH'
            if spec == 'Acetic acid':                      spec = 'ACTA'
            # Ketones
            if spec == 'Acetone':                          spec = 'ACET'
            if spec == 'MEK':                              spec = 'MEK'
            # Biogenic related compounds
            if spec == 'Isoprene':                         spec = 'ISOP'
            #if spec == 'Monoterpenes':                     spec = 'MTPA'
            if spec == 'Methacrolein':                     spec = 'MACR'
            if spec == 'MVK':                              spec = 'MVK'
            # Furans related
            if spec == 'Furan':                            spec = 'FURAN'
            if spec == 'Methylfuran':                      spec = 'M2FURAN'
            if spec == 'Dimethylfuran':                    spec = 'DIMEFURAN'
            if spec == 'Furfural':                         spec = 'FURFURAL'
            if spec == 'Methylfurfural':                   spec = 'MEFURFURAL'
            if spec == 'Maleic Anhydride':                 spec = 'MALANHY'
            
            # Unknown 
            if spec == 'PPN':                              spec = 'PPN'
            # Non-OVOC species
            if spec == 'CO':                               spec = 'CO'
            if spec == 'Benzene':                          spec = 'BENZ'
            if spec == 'O3':                               spec = 'O3'
            if spec == 'NO':                               spec = 'NO'
            if spec == 'NO2':                              spec = 'NO2'
            if spec == 'PAN':                              spec = 'PAN'
            if spec == 'HONO':                             spec = 'HNO2'
            if spec == 'HNO3':                             spec = 'HNO3'
            # Radicals
            if spec == 'OH':                               spec = 'OH'
            if spec == 'HO2':                              spec = 'HO2'
            if spec == 'CH3O2':                            spec = 'MO2'
            if spec == 'MALDIALO':                         spec = 'DIALO'
            if spec == 'MALDIALO2':                        spec = 'DIALO2'
            if spec == 'MALDIALCO2':                       spec = 'DIALCO2'
        if MCM_setting==True:
            # map to the MCM mech name, customized
            # Aldehydes
            if spec == 'Formaldehyde':          spec = 'HCHO'
            if spec == 'Acetaldehyde':          spec = 'CH3CHO'
            #if spec == 'Lumped C>=3 aldehydes': lumped them later in read in file section
            if spec == 'Glyoxal':               spec = 'GLYOX'
            if spec == 'Methylglyoxal':         spec = 'MGLYOX'
            if spec == 'Glycoaldehyde':         spec = 'HOCH2CHO'
            # Organic acids
            if spec == 'Formic acid':           spec = 'HCOOH'
            if spec == 'Acetic acid':           spec = 'CH3CO2H'
            # Ketones
            if spec == 'Acetone':               spec = 'CH3COCH3'
            if spec == 'MEK':                   spec = 'MEK'
            # Biogenic related compounds
            if spec == 'Isoprene':              spec = 'C5H8'
            #if spec == 'Monoterpenes': : lumped them later in read in file section       
            if spec == 'Methacrolein':          spec = 'MACR'
            if spec == 'MVK':                   spec = 'MVK'
            # Furans related
            if spec == 'Furan':                 spec = 'FURAN'
            if spec == 'Methylfuran':           spec = 'M2FURAN'
            if spec == 'Dimethylfuran':         spec = 'DIMEFURAN'
            if spec == 'Furfural':              spec = 'FURFURAL'
            if spec == 'Methylfurfural':        spec = 'MEFURFURAL'
            if spec == 'Maleic Anhydride':      spec = 'MALANHY'
            # Unknown  
            if spec == 'PPN':                   spec = 'PPN'
            # Non-OVOC species
            if spec == 'CO':                    spec = 'CO'
            if spec == 'Benzene':               spec = 'BENZENE'
            if spec == 'O3':                    spec = 'O3'
            if spec == 'NO':                    spec = 'NO'
            if spec == 'NO2':                   spec = 'NO2'
            if spec == 'PAN':                   spec = 'PAN'
            if spec == 'HONO':                  spec = 'HONO'
            if spec == 'HNO3':                  spec = 'HNO3'
            # Radicals
            if spec == 'OH':                    spec = 'OH'
            if spec == 'HO2':                   spec = 'HO2'
            if spec == 'CH3O2':                 spec = 'CH3O2'
            if spec == 'MALDIALO':              spec = 'MALDIALO'
            if spec == 'MALDIALO2':             spec = 'MALDIALO2'
            if spec == 'MALDIALCO2':            spec = 'MALDIALCO2'

        # =============
        # read in file
        # =============
        if obs_setting==True:
            spec=spec

        if get_conc==True:
            # read in concentration
            # lumped species
            # NOx
            if spec == 'NOx':
                try:
                    spec_conc = getF0AM_mat_conc(file_name, 'NO') + getF0AM_mat_conc(file_name, 'NO2')
                except:
                    spec_conc = [0]*len(time)
                    print('NO ' + spec + ' EXIST!')
            # HOx
            elif spec == 'HOx':
                try:
                    spec_conc = getF0AM_mat_conc(file_name, 'OH') + getF0AM_mat_conc(file_name, 'HO2')
                except:
                    spec_conc = [0]*len(time) 
                    print('NO ' + spec + ' EXIST!')
            # Lumped C>=3 aldehydes
            elif spec == 'Lumped C>=3 aldehydes':
                try:
                    spec_conc = getF0AM_mat_conc(file_name, 'C2H5CHO') + getF0AM_mat_conc(file_name, 'C3H7CHO') + \
                                    getF0AM_mat_conc(file_name, 'IPRCHO') + getF0AM_mat_conc(file_name, 'BUT2CHO') if MCM_setting else getF0AM_mat_conc(file_name, 'RCHO')
                except:
                    spec_conc = [0]*len(time) 
                    print('NO ' + spec + ' EXIST!')
            # Monoterpenes
            elif spec == 'Monoterpenes':
                try:
                    spec_conc = getF0AM_mat_conc(file_name, 'APINENE') + getF0AM_mat_conc(file_name, 'BPINENE')   if MCM_setting else getF0AM_mat_conc(file_name, 'MTPA')
                except:
                    spec_conc = [0]*len(time) 
                    print('NO ' + spec + ' EXIST!')
            # Other compounds
            else:
                try:
                    spec_conc = getF0AM_mat_conc(file_name, spec)
                except:
                    spec_conc = [0]*len(time)
                    print('NO ' + spec + ' EXIST!')

                    
        if get_bkg==True:
            # read in concentration
            # set if statement for lumped species (customized)
            # NOx
            if spec == 'NOx':
                try:
                    spec_conc = getF0AM_mat_bkg(file_name, 'NO') + getF0AM_mat_bkg(file_name, 'NO2')
                except:
                    spec_conc = [0]*len(time)
            # HOx
            elif spec == 'HOx':
                try:
                    spec_conc = getF0AM_mat_bkg(file_name, 'OH') + getF0AM_mat_bkg(file_name, 'HO2')
                except:
                    spec_conc = [0]*len(time)
            # Lumped C>=3 aldehydes
            elif spec == 'Lumped C>=3 aldehydes':
                try:
                    spec_conc = getF0AM_mat_bkg(file_name, 'C2H5CHO') + getF0AM_mat_bkg(file_name, 'C3H7CHO') + \
                                    getF0AM_mat_bkg(file_name, 'IPRCHO') + getF0AM_mat_bkg(file_name, 'BUT2CHO') if MCM_setting else getF0AM_mat_bkg(file_name, 'RCHO')
                except:
                    spec_conc = [0]*len(time) 
            # Monoterpenes
            elif spec == 'Monoterpenes':
                try:
                    spec_conc = getF0AM_mat_bkg(file_name, 'APINENE') + getF0AM_mat_bkg(file_name, 'BPINENE')  if MCM_setting else getF0AM_mat_bkg(file_name, 'MTPA')
                except:
                    spec_conc = [0]*len(time) 
            # Other compounds
            else:
                try:
                    spec_conc = getF0AM_mat_bkg(file_name, spec)
                except:
                    spec_conc = [0]*len(time)

        df_conc = pd.DataFrame(np.array(spec_conc), index = time, columns = [spec_list[ind]])
        df_dummy = pd.concat([df_dummy, df_conc], axis=1)
    return df_dummy

In [ ]:
# function to read observation data for listed species 
def get_obs_df(Fligt_ID, spec_list, get_smk, get_bkg):
    # decide which data we will read
    if get_smk: setting = 'smk'
    if get_bkg:  setting = 'bkg'
    
    # read observation data
    df_obs = pd.read_csv('/glade/work/lixujin/PYTHON/F0AM/WE-CAN/Model_inputs_prepared/output_data/' + Flight_ID + '_' + setting + '_conc.csv')
    df_obs = df_obs.set_index('Age_physical_avg_min')

    # Apply ratio for some species
    # Pay attention on the first character: Acetone, Acetic acid, mek, Methylfurfural, Hydroxyacetone, and Heptanal are processed name 
    # Aldehydes
    df_obs[setting + '_Propanal_C3H6O_PTR']                     = df_obs[setting + '_ACETONE_C3H6O_PTR']*0.17
    df_obs[setting + '_Butanal_C4H8O_PTR']                      = df_obs[setting + '_MEK_C4H8O_PTR']*0.01
    df_obs[setting + '_2_Methylpropanal_C4H8O_PTR']             = df_obs[setting + '_MEK_C4H8O_PTR']*0.14
    df_obs[setting + '_2_Methylbutanal_PTR']                    = df_obs[setting + '_3_methyl_2_butanone_C5H10O_PTR']*0.04
    # Aromatics
    df_obs[setting + '_Ethyl_benzene_C8H10_PTR']                = df_obs[setting + '_C8_AROMATICS_C8H10_PTR']*0.36
    df_obs[setting + '_mpXylenes_C8H10_PTR']                    = df_obs[setting + '_C8_AROMATICS_C8H10_PTR']*0.46
    df_obs[setting + '_oXylene_C8H10_PTR']                      = df_obs[setting + '_C8_AROMATICS_C8H10_PTR']*0.18
    # Acid
    df_obs[setting + '_Acetic_acid_C2H4O2_PTR']                 = df_obs[setting + '_acetic_acid_C2H4O2_PTR']*0.67
    # Biogenic VOCs
    df_obs[setting + '_apinene_C10H16_PTR']                     = df_obs[setting + '_MONOTERPENES_C10H16_PTR']*0.33
    df_obs[setting + '_bpinene_C10H16_PTR']                     = df_obs[setting + '_MONOTERPENES_C10H16_PTR']*0.21
    df_obs[setting + '_MVK_C4H6O_PTR']                          = df_obs[setting + '_MACR_MVK_C4H6O_PTR']*0.60
    df_obs[setting + '_MACR_C4H6O_PTR']                         = df_obs[setting + '_MACR_MVK_C4H6O_PTR']*0.28
    # Ketones
    df_obs[setting + '_Acetone_C3H6O_PTR']                      = df_obs[setting + '_ACETONE_C3H6O_PTR']*0.83
    df_obs[setting + '_mek_C4H8O_PTR']                          = df_obs[setting + '_MEK_C4H8O_PTR']*0.85
    # Furans
    df_obs[setting + '_2_Methylfuran_C5H6O_PTR']                = df_obs[setting + '_2_METHYLFURAN_C5H6O_PTR']*0.84
    df_obs[setting + '_3_Methylfuran_C5H6O_PTR']                = df_obs[setting + '_2_METHYLFURAN_C5H6O_PTR']*0.16
    df_obs[setting + '_5_Methylfurfural_C6H6O2_PTR']            = df_obs[setting + '_5_METHYLFURFURAL_C6H6O2_PTR']*0.5
    # Others
    df_obs[setting + '_glycolaldehyde_C2H4O2_PTR']              = df_obs[setting + '_acetic_acid_C2H4O2_PTR']*0.33
    df_obs[setting + '_Hydroxyacetone_C3H6O2_PTR']              = df_obs[setting + '_hydroxyacetone_C3H6O2_PTR']*0.51
    df_obs[setting + '_Methyl_acetate_C3H6O2_PTR']              = df_obs[setting + '_hydroxyacetone_C3H6O2_PTR']*0.35
    df_obs[setting + '_Ethyl_formate_C3H6O2_PTR']               = df_obs[setting + '_hydroxyacetone_C3H6O2_PTR']*0.14
    df_obs[setting + '_2_Butenal_C4H6O_PTR']                    = df_obs[setting + '_MACR_MVK_C4H6O_PTR']*0.13
    df_obs[setting + '_iBuONO2_TOGA']                           = df_obs[setting + '_iBuONO2and2BuONO2_TOGA']*0.5
    df_obs[setting + '_nBuONO2_TOGA']                           = df_obs[setting + '_iBuONO2and2BuONO2_TOGA']*0.5
    df_obs[setting + '_MBO_C5H10O_PTR']                         = df_obs[setting + '_3_methyl_2_butanone_C5H10O_PTR']*0.43
    df_obs[setting + '_2_Pentanone_C5H10O_PTR']                 = df_obs[setting + '_3_methyl_2_butanone_C5H10O_PTR']*0.23
    df_obs[setting + '_3_Pentanone_C5H10O_PTR']                 = df_obs[setting + '_3_methyl_2_butanone_C5H10O_PTR']*0.21
    df_obs[setting + '_Catechol_C6H6O2_PTR']                    = df_obs[setting + '_5_METHYLFURFURAL_C6H6O2_PTR']*0.5
    df_obs[setting + '_Hexanones_C6H12O_PTR']                   = df_obs[setting + '_3_HEXANONE_C6H12O_PTR']*0.53
    df_obs[setting + '_Hexanal_C6H12O_PTR']                     = df_obs[setting + '_3_HEXANONE_C6H12O_PTR']*0.47
    df_obs[setting + '_Heptanal_C7H14O_PTR']                    = df_obs[setting + '_heptanal_C7H14O_PTR']*0.63
    df_obs[setting + '_heptanone_C7H14O_PTR']                   = df_obs[setting + '_heptanal_C7H14O_PTR']*0.37
    # mapping to retrieve observation data (customized)
    spec_obs = []
    for spec in spec_total:
        # Aldehydes
        if spec == 'Formaldehyde':                 spec_obs.append(setting + '_formaldehyde_CH2O_PTR')
        if spec == 'Acetaldehyde':                 spec_obs.append(setting + '_ACETALDEHYDE_C2H4O_PTR')
        if spec == 'Lumped C>=3 aldehydes':        spec_obs.append(setting + '_RCHO')                          # (lumped noted)
        if spec == 'Glyoxal':                      spec_obs.append(setting + '_glyoxal_C2H2O2_PTR')
        if spec == 'Methylglyoxal':                spec_obs.append(setting + '_methyl_glyoxal_C3H4O2_PTR')
        if spec == 'Glycoaldehyde':                spec_obs.append(setting + '_glycolaldehyde_C2H4O2_PTR')
        # Organic acids
        if spec == 'Formic acid':                  spec_obs.append(setting + '_CH2O2_UWCIMS')
        if spec == 'Acetic acid':                  spec_obs.append(setting + '_Acetic_acid_C2H4O2_PTR')        # isomer noted
        # Ketones
        if spec == 'Acetone':                      spec_obs.append(setting + '_Acetone_C3H6O_PTR')             # isomer noted
        if spec == 'MEK':                          spec_obs.append(setting + '_mek_C4H8O_PTR')                 # isomer noted
        # Biogenic related compounds
        if spec == 'Isoprene':                     spec_obs.append(setting + '_ISOPRENE_C5H8_PTR') 
        if spec == 'Monoterpenes':                 spec_obs.append(setting + '_MTPA')                          # (lumped noted)
        if spec == 'Methacrolein':                 spec_obs.append(setting + '_MACR_C4H6O_PTR')                # isomer noted
        if spec == 'MVK':                          spec_obs.append(setting + '_MVK_C4H6O_PTR')                 # isomer noted
        # Furans
        if spec == 'Furan':                        spec_obs.append(setting + '_FURAN_C4H4O_PTR')
        if spec == 'Methylfuran':                  spec_obs.append(setting + '_2_Methylfuran_C5H6O_PTR')
        if spec == 'Dimethylfuran':                spec_obs.append(setting + '_2_5_dimethyl_furan_C6H8O')
        if spec == 'Furfural':                     spec_obs.append(setting + '_2_FURALDEHYDE_C5H4O2_PTR')
        if spec == 'Methylfurfural':               spec_obs.append(setting + '_5_Methylfurfural_C6H6O2_PTR')
        if spec == 'Maleic Anhydride':             spec_obs.append(setting + '_maleic_anhydride_C4H2O3_PTR')
        # Unknown 
        if spec == 'PPN':                          spec_obs.append(setting + '_PPN')
        # Non-OVOC species
        if spec == 'CO':                           spec_obs.append(setting + '_CO_QCL')
        if spec == 'Benzene':                      spec_obs.append(setting + '_BENZENE_C6H6_PTR')
        if spec == 'O3':                           spec_obs.append(setting + '_O3')
        if spec == 'NO':                           spec_obs.append(setting + '_NO')
        if spec == 'NO2':                          spec_obs.append(setting + '_NO2')
        if spec == 'NOx':                          spec_obs.append(setting + '_NOx')
        if spec == 'PAN':                          spec_obs.append(setting + '_PAN')
        if spec == 'HONO':                         spec_obs.append(setting + '_HONO_UWCIMS')
        if spec == 'HNO3':                         spec_obs.append(setting + '_HNO3_UWCIMS')
        
        # Radicals
        if spec == 'HO':                           spec_obs.append(setting + '_HO')
        if spec == 'HO2':                          spec_obs.append(setting + '_HO2')
        if spec == 'HOx':                          spec_obs.append(setting + '_HOx')
        if spec == 'CH3O2':                        spec_obs.append(setting + '_CH3O2')
        if spec == 'MALDIALO':                     spec_obs.append(setting + '_MALDIALO')
        if spec == 'MALDIALO2':                    spec_obs.append(setting + '_MALDIALO2')
        if spec == 'MALDIALCO2':                   spec_obs.append(setting + '_MALDIALCO2')
        
    # add radicals and lumped compounds (customized)
    # radicals    
    df_obs[setting + '_OH']  = [0]*len(df_obs)
    df_obs[setting + '_HO2'] = [0]*len(df_obs)
    df_obs[setting + '_HOx'] = [0]*len(df_obs)
    df_obs[setting + '_CH3O2'] = [0]*len(df_obs)
    df_obs[setting + '_MALDIALO'] = [0]*len(df_obs)
    df_obs[setting + '_MALDIALO2'] = [0]*len(df_obs)
    df_obs[setting + '_MALDIALCO2'] = [0]*len(df_obs)

    # lumped compounds
    df_obs[setting + '_NOx'] = (df_obs[setting + '_NO'] + df_obs[setting + '_NO2']).values
    df_obs[setting + '_RCHO'] = (df_obs[setting + '_Propanal_C3H6O_PTR'] + df_obs[setting + '_Butanal_C4H8O_PTR'] + 
                                 df_obs[setting + '_2_Methylpropanal_C4H8O_PTR'] + df_obs[setting + '_2_Methylbutanal_PTR'] + 
                                 df_obs[setting + '_Hexanal_C6H12O_PTR'] + df_obs[setting + '_Heptanal_C7H14O_PTR']).values
    df_obs[setting + '_MTPA'] = (df_obs[setting + '_apinene_C10H16_PTR'] + df_obs[setting + '_bpinene_C10H16_PTR']).values

    # get reduced name
    name_obs2std = {}
    for ind, name_obs in enumerate(spec_obs):
        name_obs2std[spec_obs[ind]] = spec_total[ind]
    df_obs_reduced = df_obs[spec_obs].rename(columns=name_obs2std)
    
    # change plume age from min to hour
    df_obs_reduced.index = df_obs_reduced.index#, miniute for FIREX /60
    
    return df_obs_reduced


In [ ]:
# function to interpolate two different-size dataframe
def interp(df, standard_index):
    """Return a new DataFrame with all columns values interpolated
    to the standard_index values."""
    standard_index = standard_index.astype(float)
    df_out = pd.DataFrame(index=standard_index)
    df_out.index.name = df.index.name
    old_index = np.array(df.index, dtype='float64')

    for colname, col in df.iteritems():
        col = np.array(col, dtype='float64')
        df_out[colname] = np.interp(standard_index, old_index, col)

    return df_out


### Main text for coding

In [ ]:
# ================
# Main content
# ================
# species list (customized) 
#spec_total = ['Maleic Anhydride', 'O3', 'Glyoxal', 'Methylglyoxal', 'OH', 'HO2', 'CH3O2', 'PAN', 'NOx']
#spec_total = ['Maleic Anhydride', 'O3', 'Glyoxal', 'Methylglyoxal', 'HOx', 'CH3O2', 'PAN', 'NO', 'NO2']
# spec_total = ['Maleic Anhydride', 'O3', 'Glyoxal', 'Methylglyoxal', 'Acetic acid', 'HOx', 'CH3O2', 'PAN', 'NOx']
#spec_total = ['Maleic Anhydride', 'O3', 'Glyoxal', 'Methylglyoxal', 'OH', 'HO2', 'PAN', 'NOx', 'HCOOH']
#spec_total = ['MALDIALO', 'MALDIALO2', 'MALDIALCO2']
#spec_total = ['Maleic Anhydride', 'Formaldehyde', 'Glyoxal', 'Methylglyoxal',
#              'O3', 'NOx', 'Benzene', 'CO', 'CH3O2', 
#              'PAN', 'HOx', 'Formic acid']


# For proposal: O3, HO2, CH3O2, NOx (NO + NO2), PAN, Formaldehyde, Glyoxal, Methylglyoxal, Maleic Anhydride.
#spec_total = ['O3', 'NOx', 'PAN', 'HONO',
#              'Formaldehyde', 'Acetaldehyde',
#              'Glyoxal',  'Methylglyoxal', 
#              'Acetone', 'MEK', 'Formic acid', 'Acetic acid', 'CO', 'Benzene',
#              'Furan', 'Methylfuran', 'Dimethylfuran', 'Furfural', 'Methylfurfural', 'Maleic Anhydride']

spec_total = ['Formaldehyde', 'Acetaldehyde', 'Lumped C>=3 aldehydes',
              'Glyoxal', 'Methylglyoxal', 'Glycoaldehyde',
              'Formic acid',  'Acetic acid', 
              'Acetone', 'MEK', 
              'Isoprene', 'Monoterpenes', 'Methacrolein', 'MVK',
              'Furan', 'Methylfuran', 'Dimethylfuran', 'Furfural', 'Methylfurfural', 'Maleic Anhydride',
              'CO', 'O3', 'HNO3', 'NO','NO2', 'NOx', 'PAN', 'HONO', 'PPN']

# data directory, need to be customized
Flight_ID = ''

id2fire_name = {'FIREX 2016 laboratory experiment'}

#Runing_mech = ['GC_base', 'GC_basenofixed', 'GC_modified', 
#               'GC_modified_v2', 
#               'MCM_base', 'MCM_base_modified',  'MCM_base_noFUR',
#               'MCM_GCvocs', 'MCM_GCvocs_modified'] 
Runing_mech = ['GC_base', 'GC_modified'] 

file_dir = '/glade/u/home/lixujin/matlab/F0AM-4.2.1/Setups/Examples/Lixu/FIREX_LAB/output_data/FIREX_LGPlume'

GC_base =  file_dir + 'GCv13_base' + Flight_ID + '.mat'
GC_basenofixed = file_dir + 'GCv13_base_notfixed' + Flight_ID + '.mat'
GC_modified = file_dir + 'GCv13_JINF' + Flight_ID + '.mat'
GC_modified_v2 = file_dir + 'GCv13_JINF_HCOOH' + Flight_ID + '.mat'

MCM_base_noFUR = file_dir + 'MCMv331_noFUR' + Flight_ID + '.mat'
MCM_base_FUR = file_dir + 'MCMv331_base' + Flight_ID + '.mat'
MCM_GCvocs = file_dir + 'MCMv331_GCvocs' + Flight_ID + '.mat'
MCM_GCvocs_modified = file_dir + 'MCMv331_GCvocsF' + Flight_ID + '.mat'

# setting for smk or bkg
get_smk      = True
get_smk_norm = True
get_bkg      = False
get_nemr     = False

# choose the plotting type
tsplot = True
scplot = True

start_ind = 0

In [ ]:
# =========
# load file
# =========
F0AM_model_file = '/glade/u/home/lixujin/matlab/F0AM-4.2.1/Setups/Examples/Lixu/FIREX_LAB/LagrangianPlumeData_Extrapolated.mat'
mod_struc = scipy.io.loadmat(F0AM_model_file)
# ==============
# Variable names
# ==============
var_names = []
# varaible in the data
dtype_names = [('P', 'O'), ('T', 'O'), ('TIME', 'O'), ('RH', 'O'), ('SZA', 'O'), ('CH4', 'O'), ('NO', 'O'), ('NO2', 'O'), ('O3', 'O'), ('CO2', 'O'), ('CO', 'O'), ('HCHO', 'O'), ('HONO', 'O'), ('CH3CHO', 'O'), ('C3H6', 'O'), ('BENZENE', 'O'), ('FURAN', 'O'), ('C5H8', 'O'), ('CH3COCH3', 'O'), ('HCOOH', 'O'), ('CH3CO2H', 'O'), ('BIACET', 'O'), ('MVK', 'O'), ('MGLYOX', 'O'), ('FURFURAL', 'O'), ('ACETOL', 'O'), ('JNO2', 'O'), ('CH3OH', 'O'), ('DIL1', 'O'), ('jcorr', 'O'), ('Sol', 'O'), ('MEFURAN', 'O'), ('MALANHY', 'O'), ('C4H4O3', 'O'), ('FURANONE', 'O'), ('PHENOL', 'O'), ('CATECHOL', 'O'), ('STYRENE', 'O'), ('CRESOL', 'O'), ('TIME_EXT', 'O'), ('CO_EXT', 'O'), ('H2O', 'O')]
for dname, tmp in dtype_names:
    var_names.append(dname)
# remove the inconsistent data
var_names.remove(var_names[30])
# ==============
# Variable data
# ==============
# retrieve the irregular information first
loc_date = mod_struc['DAQ'].item()[30]
lat, lon, alt = loc_date.item()[0][0][0], loc_date.item()[1][0][0], loc_date.item()[2][0][0]
startTime, updateTime =  loc_date.item()[3][0][0], loc_date.item()[4][0][0]
# other data
var_data = []
ct = 0
for ele in mod_struc['DAQ'].item():
    if ct != 30: 
        ele = [item for sublist in ele for item in sublist]
        var_data.append(ele)
    # increment the pointer
    ct += 1 

FileNotFoundError: [Errno 2] No such file or directory: '/glade/u/home/lixujin/matlab/F0AM-4.2.1/Setups/Examples/Lixu/FIREX_LAB/LagrangianPlumeData_Extrapolated.mat'

In [8]:
#InitTime = (float(df_smk.index[-1])-float(df_smk.index[start_ind]))*60/len(df_smk.index)
#InitTime

In [23]:
# ================================================
# Prepare met data
# Only for model input but not for data analysis 
# ================================================
# Met data: get ind where len of element is 65
ind_65dp = [len(num_dp) == 65 for num_dp in var_data]
met_data  = np.array(var_data)[ind_65dp] 
met_names = np.array(var_names)[ind_65dp] 
df_met = pd.DataFrame(list(met_data), index = met_names).transpose()
met_time = df_met['TIME_EXT']
df_met.index = met_time/60 # sec to min
df_met = df_met.drop('TIME_EXT', axis = 1)

# smk data: get ind where len of element is 15
ind_15dp = [len(num_dp) == 15 for num_dp in var_data]
smk_data  = np.array(var_data)[ind_15dp] 
smk_names = np.array(var_names)[ind_15dp] 
df_smk = pd.DataFrame(list(smk_data), index = smk_names).transpose()
smk_time = df_smk['TIME']
df_smk.index = smk_time/60 # sec to min

# rename the column names
df_smk = df_smk.rename(columns={"HCHO":"Formaldehyde", "CH3CHO":"Acetaldehyde", "BENZENE":"Benzene", 
                                "FURAN":"Furan", "C5H8": "Isoprene", "CH3COCH3": "Acetone", 
                               "HCOOH": "Formic acid", "CH3CO2H": "Acetic acid", "BIACET": "Biacetyl",
                               "MGLYOX": "Methylglyoxal", "FURFURAL": "Furfural", "ACETOL": "Hydroxyacetone", 
                               "CH3OH": "Methanol", "MEFURAN": "Methylfuran", "MALANHY": "Maleic Anhydride", 
                               "C4H4O3": "5-hydroxy-2(5H)-furanone", "FURANONE": "Furanone", "PHENOL": "Phenol", 
                               "CATECHOL": "Catechol", "STYRENE": "Styrene", "CRESOL": "Cresol"})

df_smk = df_smk.drop(['TIME', 'DIL1', ], axis = 1)

In [30]:
df_smk.columns

Index(['TIME', 'CH4', 'NO', 'NO2', 'O3', 'CO2', 'CO', 'HCHO', 'HONO', 'CH3CHO',
       'C3H6', 'BENZENE', 'FURAN', 'C5H8', 'CH3COCH3', 'HCOOH', 'CH3CO2H',
       'BIACET', 'MVK', 'MGLYOX', 'FURFURAL', 'ACETOL', 'CH3OH', 'DIL1',
       'MEFURAN', 'MALANHY', 'C4H4O3', 'FURANONE', 'PHENOL', 'CATECHOL',
       'STYRENE', 'CRESOL'],
      dtype='object')

In [36]:
df_smk

,TIME,CH4,NO,NO2,O3,CO2,CO,Formaldehyde,HONO,Acetaldehyde,...,Methanol,DIL1,Methylfuran,Maleic Anhydride,5-hydroxy-2(5H)-furanone,Furanone,Phenol,Catechol,Styrene,Cresol
TIME,,,,,,,,,,,,,,,,,,,,,
0.016667,1.000,2021.456228,4.272573,7.258806,33.277243,9896.539391,1107.0,22.931414,1.214734,10.548477,...,23.593595,0.000000,0.691364,0.270424,0.041333,0.883303,0.685333,0.665818,0.120576,0.438576
2.251617,135.097,1980.609176,1.648442,4.724150,34.051202,6575.354479,781.0,15.511752,1.129521,7.185535,...,16.837666,1.475093,0.523800,0.212400,0.035250,0.662500,0.477100,0.392150,0.096600,0.343150
7.031050,421.863,1937.934047,0.633320,2.867997,35.280134,3306.526443,433.0,9.103382,0.774750,3.309057,...,8.222016,2.030272,0.169421,0.108263,0.014842,0.357579,0.210632,0.346474,0.025053,0.136895
11.863300,711.798,1914.937378,0.272101,1.026520,37.121620,1779.930609,299.0,5.739078,0.416652,2.241592,...,5.253356,1.657419,0.104571,0.099095,0.011714,0.294810,0.127524,0.234238,0.016190,0.106286
16.659983,999.599,1909.056755,0.229171,0.789814,37.317494,1650.747375,246.0,4.401955,0.000000,1.772752,...,4.127052,1.347297,0.077474,0.094947,0.010211,0.241000,0.106368,0.201105,0.005105,0.080737
21.301000,1278.060,1908.088559,0.213821,0.734969,36.960459,773.929432,224.0,3.876284,0.000000,1.488509,...,4.038983,1.172653,0.040000,0.113750,0.021050,0.172550,0.076850,0.145700,0.015250,0.052550
26.037833,1562.270,1908.092323,0.183478,0.617654,38.091886,1166.857655,212.0,4.344722,0.347987,1.477448,...,3.762214,1.100253,0.046211,0.155895,0.028421,0.188211,0.068211,0.108421,0.008684,0.037105
30.637333,1838.240,1905.509715,0.212605,0.723984,39.108423,1335.147992,205.0,4.215407,0.302809,1.346811,...,3.506698,1.066301,0.027579,0.143316,0.021421,0.131368,0.058158,0.096737,0.008421,0.027579
35.321333,2119.280,1906.386511,0.171397,0.693916,38.887821,805.112608,199.0,4.321235,0.111956,1.427200,...,3.473601,1.055332,0.029550,0.122450,0.016200,0.119200,0.045600,0.061300,0.005800,0.019000


In [28]:
np.array(columns).replace('CH3OH', 'Methanol')

AttributeError: 'numpy.ndarray' object has no attribute 'replace'

In [20]:
spec_total

['Formaldehyde',
 'Acetaldehyde',
 'Lumped C>=3 aldehydes',
 'Glyoxal',
 'Methylglyoxal',
 'Glycoaldehyde',
 'Formic acid',
 'Acetic acid',
 'Acetone',
 'MEK',
 'Isoprene',
 'Monoterpenes',
 'Methacrolein',
 'MVK',
 'Furan',
 'Methylfuran',
 'Dimethylfuran',
 'Furfural',
 'Methylfurfural',
 'Maleic Anhydride',
 'CO',
 'O3',
 'HNO3',
 'NO',
 'NO2',
 'NOx',
 'PAN',
 'HONO',
 'PPN']

In [18]:
# ======================
# 1) smoke concentration
# ======================
if get_smk:
    # observations
    df_obs_smk_conc      = df_smk
    co_obs_smk_conc      = df_obs_smk_conc['CO']
    standard_index = np.array(df_obs_smk_conc.index[start_ind:-1], dtype='float64')
    
    # ----------------------------------------------------------------------------------------------
    # F0AM output processing
    # 1) read data and clean up duplicate index for F0AM model out put
    # 2) remove dummy column
    # 3) correct model simulation time with initial plume age time and get requries species
    # 4) interpolate data based on standard index (observation value to get scatterplot comparison)
    # ----------------------------------------------------------------------------------------------
    
    # GEOS-Chem base
    if 'GC_base' in Runing_mech:
        print('This is for GEOS-Chem (base)')
        # step 1
        df_gc_base_smk_conc = get_df_specs(file_name=GC_base, spec_list = spec_total, 
                                       get_conc=True, get_bkg=False, 
                                       GC_setting=True, MCM_setting=False, obs_setting=False)

        df_gc_base_smk_conc = df_gc_base_smk_conc.loc[~df_gc_base_smk_conc.index.duplicated(), :]
        # step 2
        df_gc_base_smk_conc = df_gc_base_smk_conc.drop('Dummy', inplace=False, axis=1)
        # step 3
        df_gc_base_smk_conc.index = df_obs_smk_conc.index[start_ind] + df_gc_base_smk_conc.index
        co_gc_base_smk_conc = df_gc_base_smk_conc['CO']
        # step 4
        df_gc_base_smk_conc_interp = interp(df_gc_base_smk_conc, standard_index)
        co_gc_base_smk_conc_interp = df_gc_base_smk_conc_interp['CO']

    
    # GEOS-Chem nofixed
    if 'GC_basenofixed' in Runing_mech:
        print('This is for GEOS-Chem (nofixed)')
        # step 1
        df_gc_basenofixed_smk_conc = get_df_specs(file_name=GC_basenofixed, spec_list = spec_total, 
                                       get_conc=True, get_bkg=False, 
                                       GC_setting=True, MCM_setting=False, obs_setting=False)
        df_gc_basenofixed_smk_conc = df_gc_basenofixed_smk_conc.loc[~df_gc_basenofixed_smk_conc.index.duplicated(), :]
        # step 2
        df_gc_basenofixed_smk_conc = df_gc_basenofixed_smk_conc.drop('Dummy', inplace=False, axis=1)
        # step 3
        df_gc_basenofixed_smk_conc.index = df_obs_smk_conc.index[start_ind] + df_gc_basenofixed_smk_conc.index
        co_gc_basenofixed_smk_conc = df_gc_basenofixed_smk_conc['CO']
        # step 4
        df_gc_basenofixed_smk_conc_interp = interp(df_gc_basenofixed_smk_conc, standard_index)
        co_gc_basenofixed_smk_conc_interp = df_gc_basenofixed_smk_conc_interp['CO']

    # GEOS-Chem base experiment 1
    if 'GC_modified' in Runing_mech:
        print('This is for GEOS-Chem (FUR)')
        # step 1
        df_gc_modi_smk_conc = get_df_specs(file_name=GC_modified, spec_list = spec_total, 
                                       get_conc=True, get_bkg=False, 
                                       GC_setting=True, MCM_setting=False, obs_setting=False)
        df_gc_modi_smk_conc = df_gc_modi_smk_conc.loc[~df_gc_modi_smk_conc.index.duplicated(), :]
        # step 2
        df_gc_modi_smk_conc = df_gc_modi_smk_conc.drop('Dummy', inplace=False, axis=1)
        # step 3
        df_gc_modi_smk_conc.index = df_obs_smk_conc.index[start_ind] + df_gc_modi_smk_conc.index
        co_gc_modi_smk_conc = df_gc_modi_smk_conc['CO']
        # step 4
        df_gc_modi_smk_conc_interp = interp(df_gc_modi_smk_conc, standard_index)
        co_gc_modi_smk_conc_interp = df_gc_modi_smk_conc_interp['CO']
    
    
    # GEOS-Chem base experiment 2
    if 'GC_modified_v2' in Runing_mech:
        print('This is for GEOS-Chem (FUR + SW mech)')
        # step 1
        df_gc_modi_v2_smk_conc = get_df_specs(file_name=GC_modified_v2, spec_list = spec_total, 
                                          get_conc=True, get_bkg=False, 
                                          GC_setting=True, MCM_setting=False, obs_setting=False)
        df_gc_modi_v2_smk_conc = df_gc_modi_v2_smk_conc.loc[~df_gc_modi_v2_smk_conc.index.duplicated(), :]
        co_gc_modi_v2_smk_conc = df_gc_modi_v2_smk_conc['CO']
        # step 2
        df_gc_modi_v2_smk_conc = df_gc_modi_v2_smk_conc.drop('Dummy', inplace=False, axis=1)
        # step 3
        df_gc_modi_v2_smk_conc.index = df_obs_smk_conc.index[start_ind] + df_gc_modi_v2_smk_conc.index
        co_gc_modi_v2_smk_conc = df_gc_modi_v2_smk_conc['CO']
        # step 4
        df_gc_modi_v2_smk_conc_interp = interp(df_gc_modi_v2_smk_conc, standard_index)
        co_gc_modi_v2_smk_conc_interp = df_gc_modi_v2_smk_conc_interp['CO']
    
    # MCM mechanism but only with GEOS-Chem VOCs
    if 'MCM_base_noFUR' in Runing_mech:  
        print('This is for MCM + noFUR')
        # step 1
        df_mcm_base_noFUR_smk_conc = get_df_specs(file_name=MCM_base_noFUR, spec_list = spec_total, 
                                       get_conc=True, get_bkg=False, 
                                       GC_setting=False, MCM_setting=True, obs_setting=False)
        df_mcm_base_noFUR_smk_conc = df_mcm_base_noFUR_smk_conc.loc[~df_mcm_base_noFUR_smk_conc.index.duplicated(), :]
        # step 2
        df_mcm_base_noFUR_smk_conc = df_mcm_base_noFUR_smk_conc.drop('Dummy', inplace=False, axis=1)
        # step 3
        df_mcm_base_noFUR_smk_conc.index = df_obs_smk_conc.index[start_ind] + df_mcm_base_noFUR_smk_conc.index
        co_mcm_base_noFUR_smk_conc = df_mcm_base_noFUR_smk_conc['CO']
        # step 4
        df_mcm_base_noFUR_smk_conc_interp = interp(df_mcm_base_noFUR_smk_conc, standard_index)
        co_mcm_base_noFUR_smk_conc_interp = df_mcm_base_noFUR_smk_conc_interp['CO']
        
    # MCM base
    if 'MCM_base_FUR' in Runing_mech:
        print('This is for MCM + FUR')
        # step 1
        df_mcm_base_FUR_smk_conc = get_df_specs(file_name=MCM_base_FUR, spec_list = spec_total, 
                                       get_conc=True, get_bkg=False, 
                                       GC_setting=False, MCM_setting=True, obs_setting=False)
        df_mcm_base_FUR_smk_conc = df_mcm_base_FUR_smk_conc.loc[~df_mcm_base_FUR_smk_conc.index.duplicated(), :]
        # step 2
        df_mcm_base_FUR_smk_conc = df_mcm_base_FUR_smk_conc.drop('Dummy', inplace=False, axis=1)
        # step 3
        df_mcm_base_FUR_smk_conc.index = df_obs_smk_conc.index[start_ind] + df_mcm_base_FUR_smk_conc.index
        co_mcm_base_FUR_smk_conc = df_mcm_base_FUR_smk_conc['CO']    
        # step 4
        df_mcm_base_FUR_smk_conc_interp = interp(df_mcm_base_FUR_smk_conc, standard_index)
        co_mcm_base_FUR_smk_conc_interp = df_mcm_base_FUR_smk_conc_interp['CO']        

            
    # MCM mechanism but only with GEOS-Chem VOCs
    if 'MCM_GCvocs' in Runing_mech:    
        print('This is for MCM (GEOS-Chem VOCs)')
        # step 1
        df_mcm_GCvocs_smk_conc = get_df_specs(file_name=MCM_GCvocs, spec_list = spec_total, 
                                       get_conc=True, get_bkg=False, 
                                       GC_setting=False, MCM_setting=True, obs_setting=False)
        df_mcm_GCvocs_smk_conc = df_mcm_GCvocs_smk_conc.loc[~df_mcm_GCvocs_smk_conc.index.duplicated(), :]
        # step 2
        df_mcm_GCvocs_smk_conc = df_mcm_GCvocs_smk_conc.drop('Dummy', inplace=False, axis=1)
        # step 3
        df_mcm_GCvocs_smk_conc.index = df_obs_smk_conc.index[start_ind] + df_mcm_GCvocs_smk_conc.index
        co_mcm_GCvocs_smk_conc = df_mcm_GCvocs_smk_conc['CO']
        # step 4
        df_mcm_GCvocs_smk_conc_interp = interp(df_mcm_GCvocs_smk_conc, standard_index)
        co_mcm_GCvocs_smk_conc_interp = df_mcm_GCvocs_smk_conc_interp['CO']

    
    # MCM mechanism but only with GEOS-Chem VOCs and another experiment test
    if 'MCM_GCvocs_modified' in Runing_mech:    
        print('This is for MCM (GEOS-Chem VOCs + FUR)')
        # step 1
        df_mcm_GCvocs_modified_smk_conc = get_df_specs(file_name=MCM_GCvocs_modified, spec_list = spec_total, 
                                                   get_conc=True, get_bkg=False, 
                                                   GC_setting=False, MCM_setting=True, obs_setting=False)
        df_mcm_GCvocs_modified_smk_conc = df_mcm_GCvocs_modified_smk_conc.loc[~df_mcm_GCvocs_modified_smk_conc.index.duplicated(), :]
        # step 2
        df_mcm_GCvocs_modified_smk_conc = df_mcm_GCvocs_modified_smk_conc.drop('Dummy', inplace=False, axis=1)
        # step 3
        df_mcm_GCvocs_modified_smk_conc.index = df_obs_smk_conc.index[start_ind] + df_mcm_GCvocs_modified_smk_conc.index
        co_mcm_GCvocs_modified_smk_conc = df_mcm_GCvocs_modified_smk_conc['CO']
        # step 4
        df_mcm_GCvocs_modified_smk_conc_interp = interp(df_mcm_GCvocs_modified_smk_conc, standard_index)
        co_mcm_GCvocs_modified_smk_conc_interp = df_mcm_GCvocs_modified_smk_conc_interp['CO']


This is for GEOS-Chem (base)
NO FURAN EXIST!
NO M2FURAN EXIST!
NO DIMEFURAN EXIST!
NO FURFURAL EXIST!
NO MEFURFURAL EXIST!
NO MALANHY EXIST!
This is for GEOS-Chem (FUR)


In [19]:
# =================================
# 3) Normalized smoke concentration
# F0AM output processing
# do the division between X and CO
# =================================
if get_smk_norm == True:
    # intialized the norm_conc data
    df_obs_smk_norm_conc = pd.DataFrame().reindex_like(df_obs_smk_conc)
    # GEOS-Chem base
    if 'GC_base' in Runing_mech:
        df_gc_base_smk_norm_conc = pd.DataFrame().reindex_like(df_gc_base_smk_conc)
        df_gc_base_smk_norm_conc_interp = pd.DataFrame().reindex_like(df_gc_base_smk_conc_interp)
    # GEOS-Chem basenofixed
    if 'GC_basenofixed' in Runing_mech:
        df_gc_basenofixed_smk_norm_conc = pd.DataFrame().reindex_like(df_gc_basenofixed_smk_conc)
        df_gc_basenofixed_smk_norm_conc_interp = pd.DataFrame().reindex_like(df_gc_basenofixed_smk_conc_interp)
    # GEOS-Chem base experiment 1
    if 'GC_modified' in Runing_mech:
        df_gc_modi_smk_norm_conc = pd.DataFrame().reindex_like(df_gc_modi_smk_conc)
        df_gc_modi_smk_norm_conc_interp = pd.DataFrame().reindex_like(df_gc_modi_smk_conc_interp)
    # GEOS-Chem base experiment 2
    if 'GC_modified_v2' in Runing_mech:
        df_gc_modi_v2_smk_norm_conc = pd.DataFrame().reindex_like(df_gc_modi_v2_smk_conc)
        df_gc_modi_v2_smk_norm_conc_interp = pd.DataFrame().reindex_like(df_gc_modi_v2_smk_conc_interp)
        
    # MCM mechanism with out FUR chemistry
    if 'MCM_base_noFUR' in Runing_mech:
        df_mcm_base_noFUR_smk_norm_conc = pd.DataFrame().reindex_like(df_mcm_base_noFUR_smk_conc)
        df_mcm_base_noFUR_smk_norm_conc_interp = pd.DataFrame().reindex_like(df_mcm_base_noFUR_smk_conc_interp)
        
    # MCM base
    if 'MCM_base_FUR' in Runing_mech:
        df_mcm_base_FUR_smk_norm_conc = pd.DataFrame().reindex_like(df_mcm_base_FUR_smk_conc)
        df_mcm_base_FUR_smk_norm_conc_interp = pd.DataFrame().reindex_like(df_mcm_base_FUR_smk_conc_interp)

    # MCM mechanism but only with GEOS-Chem VOCs
    if 'MCM_GCvocs' in Runing_mech:
        df_mcm_GCvocs_smk_norm_conc = pd.DataFrame().reindex_like(df_mcm_GCvocs_smk_conc)
        df_mcm_GCvocs_smk_norm_conc_interp = pd.DataFrame().reindex_like(df_mcm_GCvocs_smk_conc_interp)
    # MCM mechanism but only with GEOS-Chem VOCs and another experiment test
    if 'MCM_GCvocs_modified' in Runing_mech:
        df_mcm_GCvocs_modified_smk_norm_conc = pd.DataFrame().reindex_like(df_mcm_GCvocs_modified_smk_conc)
        df_mcm_GCvocs_modified_smk_norm_conc_interp = pd.DataFrame().reindex_like(df_mcm_GCvocs_modified_smk_conc_interp)
    for col in df_obs_smk_conc.columns:
        # Observations
        df_obs_smk_norm_conc[col] = df_obs_smk_conc[col].div(co_obs_smk_conc)      
        print(col)
        # GEOS-Chem base
        if 'GC_base' in Runing_mech:
            df_gc_base_smk_norm_conc[col] = df_gc_base_smk_conc[col].div(co_gc_base_smk_conc)
            df_gc_base_smk_norm_conc_interp[col] = df_gc_base_smk_conc_interp[col].div(co_gc_base_smk_conc_interp)
        # GEOS-Chem basenofixed
        if 'GC_basenofixed' in Runing_mech:
            df_gc_basenofixed_smk_norm_conc[col] = df_gc_basenofixed_smk_conc[col].div(co_gc_basenofixed_smk_conc)
            df_gc_basenofixed_smk_norm_conc_interp[col] = df_gc_basenofixed_smk_conc_interp[col].div(co_gc_basenofixed_smk_conc_interp)
        # GEOS-Chem base experiment 1
        if 'GC_modified' in Runing_mech:
            df_gc_modi_smk_norm_conc[col] = df_gc_modi_smk_conc[col].div(co_gc_modi_smk_conc)
            df_gc_modi_smk_norm_conc_interp[col] = df_gc_modi_smk_conc_interp[col].div(co_gc_modi_smk_conc_interp)
        # GEOS-Chem base experiment 2
        if 'GC_modified_v2' in Runing_mech:
            df_gc_modi_v2_smk_norm_conc[col] = df_gc_modi_v2_smk_conc[col].div(co_gc_modi_v2_smk_conc)
            df_gc_modi_v2_smk_norm_conc_interp[col] = df_gc_modi_v2_smk_conc_interp[col].div(co_gc_modi_v2_smk_conc_interp)
        # MCM mechanism with out FUR chemistry
        if 'MCM_base_noFUR' in Runing_mech:
            df_mcm_base_noFUR_smk_norm_conc[col] = df_mcm_base_noFUR_smk_conc[col].div(co_mcm_base_noFUR_smk_conc)
            df_mcm_base_noFUR_smk_norm_conc_interp[col] = df_mcm_base_noFUR_smk_conc_interp[col].div(co_mcm_base_noFUR_smk_conc_interp)            
        # MCM base
        if 'MCM_base_FUR' in Runing_mech:
            df_mcm_base_FUR_smk_norm_conc[col] = df_mcm_base_FUR_smk_conc[col].div(co_mcm_base_FUR_smk_conc)
            df_mcm_base_FUR_smk_norm_conc_interp[col] = df_mcm_base_FUR_smk_conc_interp[col].div(co_mcm_base_FUR_smk_conc_interp)
        # MCM mechanism but only with GEOS-Chem VOCs
        if 'MCM_GCvocs' in Runing_mech:
            df_mcm_GCvocs_smk_norm_conc[col] = df_mcm_GCvocs_smk_conc[col].div(co_mcm_GCvocs_smk_conc)
            df_mcm_GCvocs_smk_norm_conc_interp[col] = df_mcm_GCvocs_smk_conc_interp[col].div(co_mcm_GCvocs_smk_conc_interp)
        # MCM mechanism but only with GEOS-Chem VOCs and another experiment test
        if 'MCM_GCvocs_modified' in Runing_mech:
            df_mcm_GCvocs_modified_smk_norm_conc[col] = df_mcm_GCvocs_modified_smk_conc[col].div(co_GCvocs_modified_smk_conc)
            df_mcm_GCvocs_modified_smk_norm_conc_interp[col] = df_mcm_GCvocs_modified_smk_conc_interp[col].div(co_GCvocs_modified_smk_conc_interp)

NO
NO2
O3
CO
HCHO


KeyError: 'HCHO'

In [ ]:
co_gc_base_smk_conc